# RunnableParallel

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()


# プロンプト1
prompt_1 = ChatPromptTemplate.from_messages(
    {
        ("system", "あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見をください"),
        ("user", "{topic}"),
    }
)

# プロンプト2
prompt_2 = ChatPromptTemplate.from_messages(
    {
        ("system", "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください"),
        ("user", "{topic}"),
    }
)

# 複数意見をまとめるプロンプト
prompt_synthesize = ChatPromptTemplate.from_messages(
    {
        ("system", "あなたは客観的AIです。2つの意見をまとめてください"),
        ("user", "楽観的意見：{optimistic_opinion}\n悲観的意見：{pessmistic_opinion}"),
    }
)

# それぞれの意見を述べるためのChain
chain_1 = prompt_1 | model | output_parser
chain_2 = prompt_2 | model | output_parser

# 最後に全体をまとめるChain
synthesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": chain_1,
            "pessmistic_opinion": chain_2,
        }
    )
    | prompt_synthesize
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "電気自動車の普及について"})
print(output)


電気自動車の普及に関する意見は、楽観的な側面と悲観的な側面の両方があります。楽観的意見では、電気自動車の普及が環境への良い影響をもたらし、持続可能な未来への第一歩であると評価されています。技術の進化により、バッテリーの性能が向上し、充電の利便性が増すことで、生活がより快適になるとの期待も寄せられています。

一方で、悲観的意見では、電気自動車の普及にはいくつかの課題があると指摘されています。充電インフラが不十分な地域が多く、長距離移動に関する問題が残ることや、バッテリーの製造や廃棄時の環境負荷が懸念されています。これにより、期待されるほどの持続可能性が実現できない可能性も指摘されています。

まとめると、電気自動車の普及には環境への貢献や技術進化といった明るい側面がある一方で、インフラの未整備や環境への負荷といった課題も存在し、今後の展望には慎重な視点が求められています。


In [4]:
# RunnableParallelは無くても自動変換される
synthesize_chain = (
    {
        "optimistic_opinion": chain_1,
        "pessmistic_opinion": chain_2,
    }
    | prompt_synthesize
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "電気自動車の普及について"})
print(output)

電気自動車の普及について、楽観的な意見と悲観的な意見があります。楽観的な見方では、電気自動車が環境に優しい選択肢として人気を集めており、技術の進歩や充電インフラの拡充により、より多くの人々が利用することで持続可能な未来が期待できるとされています。一方、悲観的な見方では、充電インフラの整備が不十分であり、長距離移動に不便を感じる人が多いことや、バッテリーの製造が環境に与える負荷、リサイクル問題などの課題が残っていると指摘されています。また、電気自動車の普及が電力需要を増加させ、化石燃料への依存が続く可能性も懸念されています。

これらをまとめると、電気自動車の普及には明るい展望がある一方で、解決すべき多くの課題も存在しており、今後の発展には慎重な取り組みが必要であるということが言えます。


# RunnablePassthrough

In [1]:
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.retrievers import TavilySearchAPIRetriever

os.environ["TAVILY_API_KEY"] = "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"

prompt = ChatPromptTemplate.from_template('''\
                                          以下の文脈だけを踏まえて質問に回答してください。

                                          文脈："""
                                          {context}
                                          """

                                          質問： {question}
                                          ''')
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
retriever = TavilySearchAPIRetriever(k=3)  # 検索結果を３件出力

In [2]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context" : retriever, "question" : RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("今日の千葉県船橋市の天気は?")
print(output)

今日の千葉県船橋市の天気は晴れです。


## assignメソッド
- Chain全体を出力をできる

In [3]:
chain = (
    {"context" : retriever, 
     "question" : RunnablePassthrough()}
    | RunnablePassthrough.assign(answer= prompt | model| StrOutputParser())
)

output = chain.invoke("今日の千葉県船橋市の天気は?")
print(output)

{'context': [Document(metadata={'title': '船橋市の今日明日の天気 - 日本気象協会 tenki.jp', 'source': 'https://tenki.jp/forecast/3/15/4510/12204/', 'score': 0.9158166, 'images': []}, page_content='船橋市の今日明日の天気 - 日本気象協会 tenki.jp tenki.jp 船橋市---/---ｘ直前に見た情報が PM2.5分布予測 雷(予報) 雨雲レーダー(過去) アメダス 雷(実況) 洗濯 服装 星空 紫外線 気象予報士のポイント解説 PM2.5分布予測 船橋市  今日\xa011月20日(水)[大安] 9℃ 6℃ | 風 | 北の風 | | 波 | 0.5m | 明日\xa011月21日(木)[赤口] 14℃ 8℃ | 風 | 北の風後北西の風 | | 波 | 0.5m | 5.9℃(前日差:-3.2℃) 北1.3m/s 紫外線  弱い 対策は特に不要 洗濯  部屋干し推奨 乾燥機を使わないと乾かないよ 星空  0 今夜は、諦めたほうがよさそう 紫外線  弱い 対策は特に不要 洗濯  部屋干し推奨 乾燥機を使わないと乾かないよ 紫外線  弱い 対策は特に不要 洗濯  部屋干し推奨 室内に干すか、乾燥機がお勧め アメダス(気温) 日付11月22日(金)11月23日(土)11月24日(日)11月25日(月)11月26日(火)11月27日(水)11月28日(木)11月29日(金)11月30日(土)天気晴晴晴晴晴時々曇雨時々曇晴時々曇晴晴時々雨気温(℃)187149146155156191219141710189降水確率10%20%10%10%40%80%40%40%60% 気象予報士のポイント解説(日直予報士) 北西部(千葉)各地の天気 北西部(千葉) 船橋市 アメダス PM2.5 1位養老渓谷千葉県 青葉 2位成田山新勝寺千葉県 色づき始め 3位亀山湖千葉県 色づき始め 4位本土寺千葉県 色づき始め 5位三島湖千葉県 青葉 PM2.5 雷(予報) 雨雲レーダー(過去) アメダス 雷(実況) 洗濯 服装 星空 紫外線 tenki.jp公式SNS tenki.jp公式アプリ tenki.jp tenki.jp 

In [11]:
import pprint
pprint.pprint(output["answer"])

'今日、2024年3月9日（土）の千葉県船橋市の天気は晴れです。最高気温は10℃、最低気温は3℃で、降水確率は午前中0%、午後10%となっています。風は北西からの風が吹いています。'
